In [1]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import keys
import statistics
import numpy as np

In [2]:
path_personal_data_processed ='../../Data/Processed/pr_personal_data.csv'
sep = '|'

In [3]:
pr_personal_data = pd.read_csv(
                path_personal_data_processed,
                sep=sep
                )
pr_personal_data.head()

,expediente,cod_plan,des_plan,anio_apertura_expediente,exp_cerrado,exp_trasladado,tipo_traslado,exp_bloqueado,abandona,anio_convocatoria_acceso,...,des_subacesso,nota_acceso,nota_admision_def,centro_escolar_acceso,sexo,fecha_nacimiento,cod_provincia,provincia,cod_municipio,municipio
0,1,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,S,S,S,N,N,2008-09,...,LOGSE,7.16,7.16,229-COLEGIO LA ASUNCIÓN (Cáceres),H,1991-05-23,10.0,CÁCERES,1.0,CÁCERES
1,2,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,S,N,N,N,N,2008-09,...,LOGSE,6.77,NaN,NaN,D,1991-03-28,40.0,SEGOVIA,1.0,SEGOVIA
2,3,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,N,N,N,N,S,2008-09,...,Ciclos formativos,7.78,NaN,NaN,H,1985-02-03,4.0,ALMERÍA,1.0,ALMERÍA
3,4,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,S,N,N,N,N,2008-09,...,Ciclos formativos,5.88,NaN,NaN,D,1988-11-09,6.0,BADAJOZ,1.0,BADAJOZ
4,5,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,N,S,I,N,S,2008-09,...,LOGSE,6.73,6.73,160-COLEGIO SAGRADA FAMILIA,D,1991-11-03,6.0,BADAJOZ,1.0,BADAJOZ


In [4]:
analys_columns = [keys.RECORD_KEY, keys.PLAN_CODE_KEY, keys.PLAN_DESCRIPTION_KEY, keys.OPEN_YEAR_PLAN_KEY,
                  keys.DROP_OUT_KEY, keys.ACCESS_CALL_KEY, keys.ACCESS_DESCRIPTION_KEY,
                  keys.FINAL_ADMISION_NOTE_KEY, keys.GENDER_KEY, keys.BIRTH_DATE_KEY,
                  keys.PROVINCE_KEY, keys.TOWN_KEY]

In [5]:
cols_before = len(pr_personal_data.columns)
col_list_before = pr_personal_data.columns
pr_personal_data = pr_personal_data[analys_columns]
cols_after = len(pr_personal_data.columns)
col_list_after = pr_personal_data.columns
print("delete columns unused to analysis = " + str( cols_before - cols_after))
print("deleted columns are :" + str(list(set(col_list_before) - set(col_list_after))))
print("final columns are: " + str(analys_columns))

delete columns unused to analysis = 12
deleted columns are :['tipo_traslado', 'sub_acceso', 'nota_acceso', 'des_subacesso', 'acceso', 'centro_escolar_acceso', 'exp_trasladado', 'anio_convocatoria_acceso', 'exp_cerrado', 'cod_provincia', 'exp_bloqueado', 'cod_municipio']
final columns are: ['expediente', 'cod_plan', 'des_plan', 'anio_apertura_expediente', 'abandona', 'convocatoria_acceso', 'des_acceso', 'nota_admision_def', 'sexo', 'fecha_nacimiento', 'provincia', 'municipio']


In [6]:
pr_personal_data.isnull().sum()

expediente                    0
cod_plan                      0
des_plan                      0
anio_apertura_expediente      0
abandona                      0
convocatoria_acceso           0
des_acceso                    0
nota_admision_def           915
sexo                          0
fecha_nacimiento              0
provincia                    84
municipio                    90
dtype: int64

In [7]:
def get_median (cod_plan):
    global dset
    return statistics.median(dset['nota_admision_def'][(dset['nota_admision_def'].notna()) & (dset['cod_plan']==cod_plan)])
    

In [8]:
global dset
dset = pr_personal_data.copy()

In [9]:
print("null values before are = " + str(pr_personal_data.isnull().sum().sum()))
pr_personal_data[keys.FINAL_ADMISION_NOTE_KEY] = pr_personal_data.apply(lambda func: get_median(func.cod_plan) if pd.isna(func[keys.FINAL_ADMISION_NOTE_KEY]) else func[keys.FINAL_ADMISION_NOTE_KEY],axis=1)
print("null values after are = " + str(pr_personal_data.isnull().sum().sum()))


null values before are = 1089
null values after are = 174


In [10]:
pr_personal_data.isnull().sum()

expediente                   0
cod_plan                     0
des_plan                     0
anio_apertura_expediente     0
abandona                     0
convocatoria_acceso          0
des_acceso                   0
nota_admision_def            0
sexo                         0
fecha_nacimiento             0
provincia                   84
municipio                   90
dtype: int64

In [11]:
col_list_before = pr_personal_data.columns
pr_personal_data['lugar_origen'] = pr_personal_data[keys.TOWN_KEY].apply(lambda func: 'MISMO_MUNICIPIO' if func == 'CÁCERES' else (func if pd.isna(func) else 'OTRO_MUNICIPIO'))
pr_personal_data['lugar_origen'] = pr_personal_data.apply(
    lambda func: func.lugar_origen if func[keys.PROVINCE_KEY] == 'CÁCERES' or 
    func[keys.PROVINCE_KEY] == 'BADAJOZ' else(func[keys.PROVINCE_KEY] if pd.isna(func[keys.PROVINCE_KEY]) else 'OTRA_COMUNIDAD'),axis=1)
pr_personal_data.drop([keys.TOWN_KEY, keys.PROVINCE_KEY],axis=1,inplace=True)
col_list_after = pr_personal_data.columns
print("deleted columns are :" + str(list(set(col_list_before) - set(col_list_after))))
print("new column is: lugar_origen" )
print("final columns are: " + str(analys_columns))

deleted columns are :['municipio', 'provincia']
new column is: lugar_origen
final columns are: ['expediente', 'cod_plan', 'des_plan', 'anio_apertura_expediente', 'abandona', 'convocatoria_acceso', 'des_acceso', 'nota_admision_def', 'sexo', 'fecha_nacimiento', 'provincia', 'municipio']


In [12]:
rows_before = len(pr_personal_data.index)
pr_personal_data.dropna(inplace=True)
rows_after = len(pr_personal_data.index)
print("remove nans = " + str( rows_before - rows_after))

remove nans = 84


In [13]:
pr_personal_data.columns

Index(['expediente', 'cod_plan', 'des_plan', 'anio_apertura_expediente',
       'abandona', 'convocatoria_acceso', 'des_acceso', 'nota_admision_def',
       'sexo', 'fecha_nacimiento', 'lugar_origen'],
      dtype='object')

In [14]:
pr_personal_data[keys.BIRTH_DATE_KEY] = pd.to_datetime(pr_personal_data[keys.BIRTH_DATE_KEY])
pr_personal_data[keys.BIRTH_DATE_KEY] = pr_personal_data[keys.BIRTH_DATE_KEY].apply(lambda func: func.year)
pr_personal_data.rename(columns={keys.BIRTH_DATE_KEY: keys.BIRTH_YEAR_KEY}, inplace=True)
anio_nacimiento_bcket_array = np.linspace(1960, 2005, 10)
pr_personal_data[keys.BIRTH_YEAR_INTERVAL_KEY] = pd.cut(
    pr_personal_data[keys.BIRTH_YEAR_KEY], anio_nacimiento_bcket_array, include_lowest=True)
pr_personal_data.drop([keys.BIRTH_YEAR_KEY], axis=1, inplace=True)
print("Get only birth year of birth date and discretize on intervals")

Get only birth year of birth date


In [16]:
pr_personal_data[keys.DROP_OUT_KEY] = pr_personal_data[keys.DROP_OUT_KEY].apply(
    lambda func: 1 if func == 'S' else 0)
print("Change format to " + keys.DROP_OUT_KEY + " feature")

Change format to abandona feature


In [17]:
path_personal_data_preanalysis ='../../Data/Pre-analysis/pre_analys_personal_data.csv'
pr_personal_data.to_csv(
    path_personal_data_preanalysis,
    sep=sep,
    index=False)